## set up

In [1089]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 
import asyncio
import aiofiles

In [1315]:
# Define a semaphore to limit concurrency
SEMAPHORE_LIMIT = 20
semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)

async def semaphore_wrapper(func, *args):
    """Wrap async function with semaphore to limit concurrency"""
    async with semaphore:
        return await func(*args)

In [2]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [3]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [4]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [5]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [6]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [2649]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "hamlet" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "theme" # character, theme

In [2650]:
# optional, but this part needs to be manually looked up
story_info = {
    "title": "Hamlet, Prince of Denmark",
    "type": "Book",
    "author": "William Shakespeare",
    "year": "1623",
    "url": "https://www.gutenberg.org/ebooks/1524",
    "image": "https://hawkherald.com/wp-content/uploads/2024/04/IMG_9984-900x1200.jpeg",
}

In [2651]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [2652]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [2653]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [2566]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [2567]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

197254
1


In [2568]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

Chunk 0


In [2569]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
num_chapters = sum([res.numChapters for res in all_res])
markers = []
for res in all_res:
    markers.extend(res.markers)

num_chapters, markers

(5, ['ACT I', 'ACT II', 'ACT III', 'ACT IV', 'ACT V'])

In [2570]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers
first_line, last_line

('THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 '[_Exeunt, bearing off the bodies, after which a peal of ordnance is shot off._]')

In [2654]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [2655]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [2573]:
# # remove any marker that doesn't include one of these words
# exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [2574]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}

if not os.path.exists(f"json/{og_story_name}"):
    os.makedirs(f"json/{og_story_name}")
with open(f"json/{og_story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [2656]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [2584]:
story_formatted[-100:] 

'help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks. '

In [2585]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(2275, 'SCENE I. Elsinore. A platform before the Castle.')

In [2586]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(40901,
 'ACT II ',
 ' right. Nay, come, let s go together. [_Exeunt._] ACT II SCENE I. A room in Polonius s house. Enter ')

In [2587]:
last_ind = story_formatted.find(last_line)
last_ind, last_line

(176671,
 '[_Exeunt, bearing off the bodies, after which a peal of ordnance is shot off._]')

In [2588]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    # print("marker", marker)
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    # print("next_marker", next_marker)
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line
    # print("formatted_last_line", formatted_last_line)

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.rfind(cur_first_line, cur_min_index)
    print("cur_first_line", repr(cur_first_line))
    print("first_line_index", first_line_index)
    print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    print("cur_last_line", repr(cur_last_line))
    print("last_line_index", last_line_index)
    print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
    
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

cur_first_line 'SCENE I. Elsinore. A platform before the Castle.'
first_line_index 2275
Snippet around first_line_index: 'SCENE I. Elsinore. A platform before the Castle. Enter Francisco and Barnardo, two sentinels. BARNARDO. Who s there? FRANCISCO. Nay, answer me. Stand and unfold yourself. BARNARDO. Long live the King!'
cur_last_line 'ACT II '
last_line_index 40901
Snippet around last_line_index: 'not lack. Let us go in together, And still your fingers on your lips, I pray. The time is out of joint. O cursed spite, That ever I was born to set it right. Nay, come, let s go together. [_Exeunt._] '
cur_first_line 'ACT II '
first_line_index 40901
Snippet around first_line_index: 'ACT II SCENE I. A room in Polonius s house. Enter Polonius and Reynaldo. POLONIUS. Give him this money and these notes, Reynaldo. REYNALDO. I will, my lord. POLONIUS. You shall do marvellous wisely, g'
cur_last_line 'ACT III '
last_line_index 73569
Snippet around last_line_index: 'd my melancholy, As he is very 

In [2589]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter ACT I
first_line: LINE 1: SCENE I. Elsinore. A platform before the Castle.
LINE 2: Enter Francisco and Barnardo, two s
last_line: hat ever I was born to set it right.
LINE 1204: Nay, come, let's go together.
LINE 1205: [_Exeunt._]


Chapter ACT II
first_line: LINE 1: SCENE I. A room in Polonius's house.
LINE 2: Enter Polonius and Reynaldo.
LINE 3: POLONIUS.

last_line: s. The play's the thing
LINE 912: Wherein I'll catch the conscience of the King.
LINE 913: [_Exit._]


Chapter ACT III
first_line: LINE 1: SCENE I. A room in the Castle.
LINE 2: Enter King, Queen, Polonius, Ophelia, Rosencrantz and
last_line: rd an end with you.
LINE 1192: Good night, mother.
LINE 1193: [_Exit Hamlet dragging out Polonius._]


Chapter ACT IV
first_line: LINE 1: SCENE I. A room in the Castle.
LINE 2: Enter King, Queen, Rosencrantz and Guildenstern.
LINE
last_line: 8: Now fear I this will give it start again;
LINE 909: Therefore let's follow.
LINE 910: [_Exeunt._]


Chapter ACT V
first_line: LINE 1

In [2590]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{og_story_name}"):
    os.makedirs(f"chapters/{og_story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter ACT I saved.
Chapter ACT II saved.
Chapter ACT III saved.
Chapter ACT IV saved.
Chapter ACT V saved.


In [2657]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [2658]:
chapters[0]

"LINE 1: SCENE I. Elsinore. A platform before the Castle.\nLINE 2: Enter Francisco and Barnardo, two sentinels.\nLINE 3: BARNARDO.\nLINE 4: Who's there?\nLINE 5: FRANCISCO.\nLINE 6: Nay, answer me. Stand and unfold yourself.\nLINE 7: BARNARDO.\nLINE 8: Long live the King!\nLINE 9: FRANCISCO.\nLINE 10: Barnardo?\nLINE 11: BARNARDO.\nLINE 12: He.\nLINE 13: FRANCISCO.\nLINE 14: You come most carefully upon your hour.\nLINE 15: BARNARDO.\nLINE 16: 'Tis now struck twelve. Get thee to bed, Francisco.\nLINE 17: FRANCISCO.\nLINE 18: For this relief much thanks. 'Tis bitter cold,\nLINE 19: And I am sick at heart.\nLINE 20: BARNARDO.\nLINE 21: Have you had quiet guard?\nLINE 22: FRANCISCO.\nLINE 23: Not a mouse stirring.\nLINE 24: BARNARDO.\nLINE 25: Well, good night.\nLINE 26: If you do meet Horatio and Marcellus,\nLINE 27: The rivals of my watch, bid them make haste.\nLINE 28: Enter Horatio and Marcellus.\nLINE 29: FRANCISCO.\nLINE 30: I think I hear them. Stand, ho! Who is there?\nLINE 31: HO

## analyze scene

In [27]:
scene_type = "character" # location, character, or n/a

In [28]:
extra_scene_instruction = "Analyze the scenes in this chapter."
if scene_type == "location":
    extra_scene_instruction += " Start a new scene when the location changes."
elif scene_type == "character":
    extra_scene_instruction += " Start a new scene when new characters enter or old characters leave."

extra_scene_instruction += "\nChapter text:"

extra_scene_instruction

'Analyze the scenes in this chapter. Start a new scene when new characters enter or old characters leave.\nChapter text:'

In [29]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [32]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A brief explanation of the character's emotions and/or actions in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A brief explanation of the theme in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [1064]:
async def get_scene_info_async(chapters, chapter_num):
    print(f"Splitting Chapter {markers[chapter_num]} into scenes...")

    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = await llm3.ainvoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = await scene_llm.ainvoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [1065]:
async def get_scene_details_async(scenes, formatted_chapter_scenes, chapter_num):
    print(f"Generating scene details for Chapter {markers[chapter_num]}...")
    all_scene_details = []

    async def process_scene(scene_text, cur_scene):
        """Async function to analyze a single scene"""
        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct quotes from the text.
                          Don't paraphrase or change the text in any way.
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a direct quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a direct quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way.
                            Try your best to find a direct quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a direct quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_theme.ainvoke(start_prompt + cur_scene)
                else:
                    scene_details = await scene_details_llm.ainvoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except:
                invalid_input = True
        
        return scene_details

    # Run all scene details in parallel
    tasks = [process_scene(formatted_chapter_scenes[i]["text"], scenes[i]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if role is not N/A and emotion is not N/A and quote is not "No quote available"
            if not (character.role == "N/A" and character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [1316]:
async def process_all_chapters_async():
    # Step 1: Split each chapter into scenes in parallel
    chapter_tasks = [get_scene_info_async(chapters, i) for i in range(len(chapters))]
    chapter_results = await asyncio.gather(*chapter_tasks)

    # extract scenes and formatted_chapter_scenes
    scenes = [result[0] for result in chapter_results]
    formatted_chapter_scenes = [result[1] for result in chapter_results]

    # Step 2: Get scene details for each scene in parallel
    detail_tasks = [semaphore_wrapper(get_scene_details_async, scenes[i], formatted_chapter_scenes[i], i) for i in range(len(chapters))]
    final_formatted_scenes = await asyncio.gather(*detail_tasks)

    # Step 3: Filter and save results
    for i, marker in enumerate(markers):
        formatted_chapter_scenes = final_formatted_scenes[i]

        # remove scenes with no text
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
        # remove characters with no quote
        for scene in formatted_chapter_scenes:
            scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
            # print(scene)
        
        # remove scenes with no characters
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

        # save as json
        with open(f"json/{story_name}/{marker}.json", "w") as f:
            json.dump(formatted_chapter_scenes, f, indent=4)

        print(f"All details for Chapter {marker} saved.\n")

In [2659]:
# RUN MAIN SCENE LOOP ASYNC that will get all the scene/character/etc. info for each chapter
await process_all_chapters_async()

Splitting Chapter ACT I into scenes...
Splitting Chapter ACT II into scenes...
Splitting Chapter ACT III into scenes...
Splitting Chapter ACT IV into scenes...
Splitting Chapter ACT V into scenes...
FIRST LINE: 1
LAST LINE: 59

FIRST LINE: 60
LAST LINE: 301

FIRST LINE: 302
LAST LINE: 585

FIRST LINE: 586
LAST LINE: 910

FIRST LINE: 612
LAST LINE: 650

FIRST LINE: 652
LAST LINE: 910

FIRST LINE: 1
LAST LINE: 39

FIRST LINE: 71
LAST LINE: 114

FIRST LINE: 228
LAST LINE: 270

FIRST LINE: 271
LAST LINE: 399

FIRST LINE: 400
LAST LINE: 681

FIRST LINE: 682
LAST LINE: 978

FIRST LINE: 1
LAST LINE: 170

FIRST LINE: 171
LAST LINE: 170

FIRST LINE: 171
LAST LINE: 223

FIRST LINE: 224
LAST LINE: 286

FIRST LINE: 287
LAST LINE: 364

FIRST LINE: 397
LAST LINE: 470

FIRST LINE: 471
LAST LINE: 554

FIRST LINE: 660
LAST LINE: 912

FIRST LINE: 1
LAST LINE: 44

FIRST LINE: 85
LAST LINE: 121

FIRST LINE: 122
LAST LINE: 174

FIRST LINE: 211
LAST LINE: 243

FIRST LINE: 244
LAST LINE: 284

FIRST LINE: 286

In [2660]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

ACT I
ACT II
ACT III
ACT IV
ACT V


In [2661]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [2662]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [2663]:
len(all_characters), len(all_locations)

(84, 27)

In [2664]:
all_characters

{'Duty and Loyalty': 'The sentinels express their commitment to their roles as guards.',
 'Isolation and Fear': 'Francisco expresses his discomfort and unease during the night watch.',
 'Friendship and Camaraderie': 'The interaction between the sentinels shows a bond of friendship and support.',
 'Supernatural Encounter': 'The appearance of the ghost sets the stage for the unfolding drama.',
 'Skepticism': "Horatio's disbelief contrasts with the others' fear and belief in the ghost.",
 'Friendship and Loyalty': 'The camaraderie among the characters highlights their shared experience and support.',
 'Supernatural Appearance': 'The ghost of the King appears, causing fear and uncertainty.',
 'Political Tension': 'Discussion of the threat posed by Fortinbras and the state of the kingdom.',
 'Historical Legacy': 'The past actions of the King and Hamlet shape the current political landscape.',
 'Grief and Mourning': "Expresses Hamlet's sorrow over his father's death and his mother's quick re

In [2665]:
all_locations

['Elsinore Castle platform',
 'The battlements of Elsinore',
 'On the battlements',
 'Elsinore Castle',
 "Polonius's house",
 'Remote part of the Castle',
 'A room in the Castle',
 "King's court",
 'Royal court of Denmark',
 "Hamlet's soliloquy",
 'A private conversation',
 "King's chamber",
 'A hall in the Castle',
 "Hamlet's chamber",
 'Danish court',
 'Theater stage',
 'A royal chamber',
 "Queen's chamber",
 'Castle room',
 'Castle in Denmark',
 'Elsinore Castle Room',
 'Castle, Various Rooms',
 'A churchyard',
 'Graveyard with Clowns',
 'Graveyard with skull',
 'Graveyard',
 'The Danish court']

In [2666]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [2667]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [2668]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [2669]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [2670]:
output

AIMessage(content='{\n    "Duty and Loyalty": ["Duty and Loyalty", "Friendship and Loyalty", "Friendship and Camaraderie"],\n    "Isolation and Fear": ["Isolation and Fear"],\n    "Supernatural": ["Supernatural Encounter", "Supernatural Appearance"],\n    "Skepticism": ["Skepticism"],\n    "Political Tension": ["Political Tension", "Political Maneuvering"],\n    "Historical Legacy": ["Historical Legacy"],\n    "Grief and Mourning": ["Grief and Mourning", "Grief", "Grief and Loss"],\n    "Corruption and Decay": ["Corruption and Decay", "Moral Corruption"],\n    "Appearance vs. Reality": ["Appearance vs. Reality", "Illusion and Reality", "Perception of Reality"],\n    "Betrayal": ["Betrayal", "Deception and Betrayal"],\n    "Parental Control": ["Parental Control", "Mother\'s Influence", "Mother-Son Conflict"],\n    "Youth and Inexperience": ["Youth and Inexperience"],\n    "Revenge": ["Revenge", "Honor and Revenge"],\n    "Guilt": ["Guilt", "Guilt and Repentance", "Guilt and Responsibili

In [2671]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [2672]:
output

{'Duty and Loyalty': ['Duty and Loyalty',
  'Friendship and Loyalty',
  'Friendship and Camaraderie'],
 'Isolation and Fear': ['Isolation and Fear'],
 'Supernatural': ['Supernatural Encounter', 'Supernatural Appearance'],
 'Skepticism': ['Skepticism'],
 'Political Tension': ['Political Tension', 'Political Maneuvering'],
 'Historical Legacy': ['Historical Legacy'],
 'Grief and Mourning': ['Grief and Mourning', 'Grief', 'Grief and Loss'],
 'Corruption and Decay': ['Corruption and Decay', 'Moral Corruption'],
 'Appearance vs. Reality': ['Appearance vs. Reality',
  'Illusion and Reality',
  'Perception of Reality'],
 'Betrayal': ['Betrayal', 'Deception and Betrayal'],
 'Parental Control': ['Parental Control',
  "Mother's Influence",
  'Mother-Son Conflict'],
 'Youth and Inexperience': ['Youth and Inexperience'],
 'Revenge': ['Revenge', 'Honor and Revenge'],
 'Guilt': ['Guilt', 'Guilt and Repentance', 'Guilt and Responsibility'],
 'Deception': ['Deception',
  'Deception and Honesty',
  'Ma

In [2673]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [2674]:
location_output

AIMessage(content='{\n  "Elsinore Castle": [\n    "Elsinore Castle platform",\n    "The battlements of Elsinore",\n    "On the battlements",\n    "Remote part of the Castle",\n    "A room in the Castle",\n    "King\'s court",\n    "Royal court of Denmark",\n    "A hall in the Castle",\n    "Hamlet\'s chamber",\n    "Danish court",\n    "A royal chamber",\n    "Queen\'s chamber",\n    "Castle room",\n    "Castle in Denmark",\n    "Elsinore Castle Room",\n    "Castle, Various Rooms",\n    "The Danish court"\n  ],\n  "Polonius\'s house": ["Polonius\'s house"],\n  "Theater stage": ["Theater stage"],\n  "Graveyard": [\n    "A churchyard",\n    "Graveyard with Clowns",\n    "Graveyard with skull"\n  ],\n  "Hamlet\'s soliloquy": ["Hamlet\'s soliloquy"],\n  "A private conversation": ["A private conversation"]\n}', additional_kwargs={}, response_metadata={'id': 'msg_01FbMk2ZaxKei3m9d6uQqyWK', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cac

In [2675]:
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [2676]:
location_output

{'Elsinore Castle': ['Elsinore Castle platform',
  'The battlements of Elsinore',
  'On the battlements',
  'Remote part of the Castle',
  'A room in the Castle',
  "King's court",
  'Royal court of Denmark',
  'A hall in the Castle',
  "Hamlet's chamber",
  'Danish court',
  'A royal chamber',
  "Queen's chamber",
  'Castle room',
  'Castle in Denmark',
  'Elsinore Castle Room',
  'Castle, Various Rooms',
  'The Danish court'],
 "Polonius's house": ["Polonius's house"],
 'Theater stage': ['Theater stage'],
 'Graveyard': ['A churchyard',
  'Graveyard with Clowns',
  'Graveyard with skull'],
 "Hamlet's soliloquy": ["Hamlet's soliloquy"],
 'A private conversation': ['A private conversation']}

In [2677]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [2678]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [2679]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [2680]:
len(all_characters_new), len(all_locations_new)

(42, 7)

In [2681]:
all_characters_new 

{'Duty And Loyalty': 3,
 'Isolation And Fear': 1,
 'Supernatural': 4,
 'Skepticism': 1,
 'Political Tension': 2,
 'Historical Legacy': 1,
 'Grief And Mourning': 6,
 'Corruption And Decay': 2,
 'Appearance Vs. Reality': 3,
 'Betrayal': 6,
 'Protective Advice': 1,
 'Parental Control': 5,
 'Youth And Inexperience': 1,
 'Revenge': 6,
 'Guilt': 3,
 'Deception': 8,
 'Manipulation': 6,
 'Madness': 11,
 'Friendship': 5,
 'Concern': 1,
 'Family Dynamics': 1,
 'Ambition And Power': 3,
 'Love': 6,
 'Existential Reflection': 4,
 'Theatricality': 7,
 'Identity': 1,
 'Mortality': 5,
 'Performance And Art': 1,
 'Fear Of Death': 1,
 'Conscience And Inaction': 1,
 'Honesty And Deceit': 1,
 'Despair And Isolation': 1,
 'Fate And Free Will': 3,
 'Honor': 2,
 'Political Turmoil': 1,
 'Death And Burial': 1,
 'Social Class': 1,
 'Morality And Guilt': 1,
 'Humor In Death': 1,
 'The Absurdity Of Life': 1,
 'Memory And Legacy': 2,
 'Conflict And Rivalry': 1}

In [2682]:

all_locations_new

['Elsinore Castle',
 "Polonius's House",
 "Hamlet's Soliloquy",
 'A Private Conversation',
 "King's Chamber",
 'Theater Stage',
 'Graveyard']

## produce chapter summaries

In [2683]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [2684]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [2685]:
async def read_json_async(file_path):
    """Asynchronously read a JSON file."""
    async with aiofiles.open(file_path, mode="r") as f:
        data = await f.read()
        return json.loads(data)
    
async def read_text_async(file_path):
    """Asynchronously read a text file."""
    async with aiofiles.open(file_path, mode="r") as f:
        return await f.read()
    
async def process_chapter_summary_async(marker):
    """Process the summary for a single chapter."""
    scenes_file = f"json/{story_name}/{marker}.json"
    chapter_file = f"chapters/{og_story_name}/{marker}.txt"

    # Read JSON and text files asynchronously
    scenes = await asyncio.gather(read_json_async(scenes_file), read_text_async(chapter_file))

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in scenes[0]])

    # Get summary from LLM
    summary = await chapter_llm.ainvoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }

    return chapter_summary

async def generate_chapter_summaries_async():
    """Generate summaries for all chapters in parallel."""
    # Processs chapters in parallel
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async(marker) for marker in markers])

    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    # Aggregate character and location information
    for scene in scenes:
        for scene in scenes:
            chapter = scene["chapter"]
            chapter_summary = chapter_summaries[markers.index(chapter)]
            # add location to chapter summary
            location = scene["location"]
            chapter_summary_locations = chapter_summary["locations"]
            if location not in chapter_summary_locations:
                chapter_summary_locations[location] = 1
            else:
                chapter_summary_locations[location] += 1
            # add importance to chapter summary
            importance = scene["importance"]
            chapter_summary["importance"].append(importance)
            # add conflict to chapter summary
            conflict = scene["conflict"]
            chapter_summary["conflict"].append(conflict)
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    # now average importance and conflict for each chapter
    for chapter_summary in chapter_summaries:
        if len(chapter_summary["importance"]) == 0:
            print(f"Chapter {chapter_summary['chapter']} has no scenes.")
            continue
        chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
        chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

    print("Importances", [chapter_summary["importance"] for chapter_summary in chapter_summaries])
    print("Conflicts", [chapter_summary["conflict"] for chapter_summary in chapter_summaries])

    # rank each chapter by importance and conflict
    importances = []
    conflicts = []
    for chapter_summary in chapter_summaries:
        importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
        conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
        
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each chaptera
    for k, (j, _) in enumerate(sorted_importances):
        chapter_summaries[markers.index(j)]["importance_rank"] = k+1

    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each chapter
    for k, (j, _) in enumerate(sorted_conflicts):
        chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

    return chapter_summaries

In [2686]:
# create summary json of each chapter
chapter_summaries = await generate_chapter_summaries_async()

Importances [0.88, 0.83, 0.86, 0.93, 0.92]
Conflicts [0.68, 0.51, 0.55, 0.85, 0.6]


In [2687]:
chapter_summaries

[{'chapter': 'ACT I',
  'scenes': 2,
  'length': 41856,
  'num_lines': 978,
  'summary': "The chapter introduces the ghost of the dead King, explores Hamlet's grief over his father's death and his mother's remarriage, and reveals the ghost's revelation about the King's murder.",
  'importance': 0.88,
  'conflict': 0.68,
  'locations': {'Elsinore Castle': 170, "Polonius's House": 34},
  'characters': {'Duty And Loyalty': 102,
   'Isolation And Fear': 34,
   'Supernatural': 136,
   'Skepticism': 34,
   'Political Tension': 34,
   'Historical Legacy': 34,
   'Grief And Mourning': 34,
   'Corruption And Decay': 34,
   'Appearance Vs. Reality': 34,
   'Betrayal': 34,
   'Protective Advice': 34,
   'Parental Control': 34,
   'Youth And Inexperience': 34,
   'Revenge': 34,
   'Guilt': 34,
   'Deception': 34},
  'importance_rank': 3,
  'conflict_rank': 2},
 {'chapter': 'ACT II',
  'scenes': 2,
  'length': 41856,
  'num_lines': 978,
  'summary': "Polonius sends Reynaldo to spy on Hamlet while t

In [2688]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [2689]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [2690]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [2691]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Madness': 11,
 'Deception': 8,
 'Theatricality': 7,
 'Grief And Mourning': 6,
 'Betrayal': 6,
 'Revenge': 6,
 'Manipulation': 6,
 'Love': 6,
 'Parental Control': 5,
 'Friendship': 5,
 'Mortality': 5,
 'Supernatural': 4,
 'Existential Reflection': 4,
 'Duty And Loyalty': 3,
 'Appearance Vs. Reality': 3,
 'Guilt': 3,
 'Ambition And Power': 3,
 'Fate And Free Will': 3,
 'Political Tension': 2,
 'Corruption And Decay': 2}

In [2692]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
2


In [2693]:
async def process_character_chunk(i, arr):
    """Process a single chunk of character assignment asynchronously."""
    invalid_input = True
    while invalid_input:
        try:
            # filter scenes to include only those with relevant characters
            new_scenes = [scene for scene in scenes if any(char["name"] in arr for char in scene["characters"])]

             # take middle 126000 characters of new_scenes
            new_scenes_str = json.dumps(new_scenes)
            new_scenes_str = new_scenes_str[:126000]

            # construct prompt
            prompt = f"""characters: {arr}
                         story info: {new_scenes_str}

                         your output should contain exactly the same {len(arr)} characters as in the original list.

                         remember to only use direct quotes from the text and not paraphrase or change the text in any way.
                         """

            # invoke model
            res = await characterlist_llm.ainvoke(prompt)

            print(f"Chunk {i} processed.")
            return res

        except:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            await asyncio.sleep(1) # wait for 1 second before trying again

async def process_all_characters():
    """Process all characters in parallel."""
    tasks = [process_character_chunk(i, arr) for i, arr in enumerate(character_arr)]
    results = await asyncio.gather(*tasks)
    return results

In [2694]:
# Run the main function to process all characters
all_res = await process_all_characters()

Chunk 2 processed.
Chunk 0 processed.
Chunk 1 processed.


In [2695]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
2


42

In [2696]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [2697]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [2698]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(18,
 ['Art',
  'Political',
  'Humor',
  'Historical Themes',
  'Death',
  'Moral',
  'Political Themes',
  'Family Themes',
  'Reflection on Death',
  'Existential',
  'Self',
  'Tension and Confrontation',
  'Social',
  'Supernatural Elements',
  'Performance',
  'Social Themes',
  'Emotions',
  'Family'])

In [2699]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [2700]:
output

AIMessage(content='{\n  "Art": ["Art", "Performance"],\n  "Political": ["Political", "Political Themes"],\n  "Humor": ["Humor"],\n  "Historical Themes": ["Historical Themes"],\n  "Death": ["Death", "Reflection on Death"],\n  "Moral": ["Moral"],\n  "Family": ["Family", "Family Themes"],\n  "Existential": ["Existential"],\n  "Self": ["Self"],\n  "Tension and Confrontation": ["Tension and Confrontation"],\n  "Social": ["Social", "Social Themes"],\n  "Supernatural Elements": ["Supernatural Elements"],\n  "Emotions": ["Emotions"]\n}', additional_kwargs={}, response_metadata={'id': 'msg_01AxkoRD22Z2uX6TsgJVmHsY', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 325, 'output_tokens': 162}}, id='run-798a4b7e-0873-46fe-af91-7ffe2652393c-0', usage_metadata={'input_tokens': 325, 'output_tokens': 162, 'total_tokens': 487, 'input_token_details': {'cache_read': 0, 'cache_

In [2701]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(13,
 {'Art': ['Art', 'Performance'],
  'Political': ['Political', 'Political Themes'],
  'Humor': ['Humor'],
  'Historical Themes': ['Historical Themes'],
  'Death': ['Death', 'Reflection on Death'],
  'Moral': ['Moral'],
  'Family': ['Family', 'Family Themes'],
  'Existential': ['Existential'],
  'Self': ['Self'],
  'Tension and Confrontation': ['Tension and Confrontation'],
  'Social': ['Social', 'Social Themes'],
  'Supernatural Elements': ['Supernatural Elements'],
  'Emotions': ['Emotions']})

In [2702]:
# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [2703]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2704]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

In [2705]:
colorinfo = {}

for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Madness': {'color': 'rgb(255, 0, 0)',
  'explanation': 'Red represents the chaotic and intense emotions associated with madness.'},
 'Deception': {'color': 'rgb(0, 128, 0)',
  'explanation': 'Green symbolizes the cunning and treachery involved in deception.'},
 'Theatricality': {'color': 'rgb(255, 165, 0)',
  'explanation': 'Orange reflects the vibrant and dynamic nature of performance and art.'},
 'Grief And Mourning': {'color': 'rgb(0, 0, 255)',
  'explanation': 'Blue conveys the deep sorrow and sadness associated with grief.'},
 'Betrayal': {'color': 'rgb(128, 0, 128)',
  'explanation': 'Purple signifies the pain and complexity of betrayal in relationships.'},
 'Revenge': {'color': 'rgb(255, 20, 147)',
  'explanation': 'Deep pink represents the passionate and intense desire for revenge.'},
 'Manipulation': {'color': 'rgb(255, 255, 0)',
  'explanation': 'Yellow symbolizes the cunning and strategic nature of manipulation.'},
 'Love': {'color': 'rgb(255, 192, 203)',
  'explanation': 

In [2706]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

42


In [2707]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(42,
 [{'name': 'Duty And Loyalty',
   'quote': 'Long live the King!',
   'group': 'Social',
   'color': 'rgb(0, 0, 139)',
   'explanation': 'Dark blue symbolizes the seriousness and commitment of duty and loyalty.'},
  {'name': 'Isolation And Fear',
   'quote': "'Tis bitter cold, And I am sick at heart.",
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Supernatural',
   'quote': 'It harrows me with fear and wonder.',
   'group': 'Supernatural Elements',
   'color': 'rgb(75, 0, 130)',
   'explanation': 'Indigo signifies the mysterious and otherworldly aspects of the supernatural.'},
  {'name': 'Skepticism',
   'quote': "'tis but our fantasy, And will not let belief take hold of him",
   'group': 'Social',
   'color': '',
   'explanation': ''},
  {'name': 'Political Tension',
   'quote': 'This bodes some strange eruption to our state.',
   'group': 'Political',
   'color': 'rgb(255, 69, 0)',
   'explanation': 'Red-orange signifies the heated and often volatile

In [2708]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(42,
 [{'name': 'Duty And Loyalty',
   'quote': 'Long live the King!',
   'group': 'Social',
   'color': 'rgb(0, 0, 139)',
   'explanation': 'Dark blue symbolizes the seriousness and commitment of duty and loyalty.'},
  {'name': 'Isolation And Fear',
   'quote': "'Tis bitter cold, And I am sick at heart.",
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Supernatural',
   'quote': 'It harrows me with fear and wonder.',
   'group': 'Supernatural Elements',
   'color': 'rgb(75, 0, 130)',
   'explanation': 'Indigo signifies the mysterious and otherworldly aspects of the supernatural.'},
  {'name': 'Skepticism',
   'quote': "'tis but our fantasy, And will not let belief take hold of him",
   'group': 'Social',
   'color': '',
   'explanation': ''},
  {'name': 'Political Tension',
   'quote': 'This bodes some strange eruption to our state.',
   'group': 'Political',
   'color': 'rgb(255, 69, 0)',
   'explanation': 'Red-orange signifies the heated and often volatile

## produce location summaries

In [2709]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [2710]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2711]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

7
[{'name': 'Elsinore Castle', 'quote': 'A bloody deed. Almost as bad, good mother, As kill a king and marry with his brother.', 'emoji': '🏰'}, {'name': "Polonius's House", 'quote': 'Fear it, Ophelia, fear it, my dear sister; and keep you in the rear of your affection, out of the shot and danger of desire.', 'emoji': '🏠'}, {'name': "Hamlet's Soliloquy", 'quote': "To be, or not to be, that is the question: Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous fortune.", 'emoji': '🗣️'}, {'name': 'A Private Conversation', 'quote': 'I did love you once.', 'emoji': '💬'}, {'name': "King's Chamber", 'quote': "Madness in great ones must not unwatch'd go.", 'emoji': '👑'}, {'name': 'Theater Stage', 'quote': "The play's the thing wherein I'll catch the conscience of the King.", 'emoji': '🎭'}, {'name': 'Graveyard', 'quote': 'Alas, poor Yorick! I knew him, Horatio: a fellow of infinite jest, of most excellent fancy.', 'emoji': '⚰️'}]


## generating the final json file!

In [2712]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Hamlet, Prince of Denmark',
 'type': 'Book',
 'author': 'William Shakespeare',
 'year': '1623',
 'url': 'https://www.gutenberg.org/ebooks/1524',
 'image': 'https://hawkherald.com/wp-content/uploads/2024/04/IMG_9984-900x1200.jpeg',
 'num_chapters': 5,
 'num_scenes': 34,
 'num_characters': 42,
 'num_locations': 7,
 'chapters': [{'chapter': 'ACT I',
   'scenes': 2,
   'length': 41856,
   'num_lines': 978,
   'summary': "The chapter introduces the ghost of the dead King, explores Hamlet's grief over his father's death and his mother's remarriage, and reveals the ghost's revelation about the King's murder.",
   'importance': 0.88,
   'conflict': 0.68,
   'locations': {'Elsinore Castle': 170, "Polonius's House": 34},
   'characters': {'Duty And Loyalty': 102,
    'Isolation And Fear': 34,
    'Supernatural': 136,
    'Skepticism': 34,
    'Political Tension': 34,
    'Historical Legacy': 34,
    'Grief And Mourning': 34,
    'Corruption And Decay': 34,
    'Appearance Vs. Reality'

In [2713]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)